### Task

In [7]:
# from dotenv import load_dotenv
# import os

# load_dotenv()
# API_KEY = os.getenv("OPENWEATHER_API_KEY")
# API_KEY

None


In [1]:
#### TEST
import requests
import pandas as pd
from datetime import datetime
import time
import pytz
from datetime import timedelta

import nest_asyncio
import asyncio
import aiohttp
from prefect import flow, task # Prefect flow and task decorators
from math import ceil

from dotenv import load_dotenv
import os

nest_asyncio.apply()

load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")

@task
async def fetch_pollution_data(df_sample, dt, localtime, batch_size=300):  #sample
    # API_KEY = "13fd31c548dbeae77a5c0d773a99df79"
    # API_KEY = os.getenv("OPENWEATHER_API_KEY")
    # WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"

    # dt = datetime.utcnow()
    # thai_tz = pytz.timezone('Asia/Bangkok')
    # localtime = dt.astimezone(thai_tz)
    
    async def fetch_row(session, row):
        # await asyncio.sleep(1)  #พัก 1 วิแบบไม่บล็อก loop
        lat = row['lat']
        lon = row['lon']
        province = row['province_en']
        district = row['district_en']
        district_id = row['district_id']
        try:
            params = {
                "lat" : lat,
                "lon" : lon,
                "appid": API_KEY,
                "units": "metric"
            }
            
            
            async with session.get(POLLUTION_ENDPOINT, params=params) as response:
                # if response.status != 200:
                #     return {
                #         'province': province,
                #         'amphoe': amphoe,
                #         'lat': lat,
                #         'lon': lon,
                #         'error': f"HTTP {response.status}"
                #     }
                data = await response.json()
               
                components = data['list'][0]['components']
                pollution_dict = {
                    'timestamp': dt,
                    'year': dt.year,
                    'month': dt.month,
                    'day': dt.day,
                    'hour': dt.hour,
                    'minute': dt.minute,
                    'localtime': localtime,
                    'province' : province,
                    'district' : district,
                    'district_id' : district_id,
                    'lat' : data['coord']['lat'],
                    'lon' : data['coord']['lon'],
                    # 'location': data['name'],
                    'main.aqi' : data['list'][0]['main']['aqi'],
                    'components_co' : components['co'],
                    'components_no' : components['no'],
                    'components_no2' : components['no2'],
                    'components_o3' : components['o3'],
                    'components_so2' : components['so2'],
                    'components_pm2_5' : components['pm2_5'],
                    'components_pm10' : components['pm10'],
                    'components_nh3' : components['nh3']
                    }
                return pollution_dict

        # # Check Error
        # except Exception as e:
        #     return {
        #         'province': row.get('province', 'unknown'),
        #         'amphoe': row.get('amphoe', 'unknown'),
        #         'lat': row.get('lat', None),
        #         'lon': row.get('lon', None),
        #         'error': str(e)
        # }
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None
        except KeyError as e:
            print(f"Error processing data: Missing key {e}")
            return None
        except Exception as e:
            print(f"Error: {e} at {province} - {district}")
            return None



    pollution_results = []
    total_batches = ceil(len(df_sample) / batch_size)  #sample
    async with aiohttp.ClientSession() as session:
        for i in range(total_batches):
            batch = df_sample.iloc[i*batch_size:(i+1)*batch_size]
            tasks = [fetch_row(session, row) for _, row in batch.iterrows()]
            batch_results = await asyncio.gather(*tasks)
            pollution_results.extend(batch_results)

            print(f"✅ เสร็จ batch {i+1}/{total_batches}")
            if i < total_batches - 1:
                await asyncio.sleep(65)  # ✅ รอให้ผ่าน rate limit

    return pollution_results



@task
async def fetch_weather_data(df_sample, dt, localtime, batch_size=300):  #sample
    # API_KEY = "13fd31c548dbeae77a5c0d773a99df79"
    # API_KEY = os.getenv("OPENWEATHER_API_KEY")
    WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    # POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
    
    # dt = datetime.utcnow()
    # thai_tz = pytz.timezone('Asia/Bangkok')
    # localtime = dt.astimezone(thai_tz)

    
    async def fetch_row(session, row):
        # await asyncio.sleep(1)  #พัก 1 วิแบบไม่บล็อก loop
        lat = row['lat']
        lon = row['lon']
        province = row['province_en']
        district = row['district_en']
        district_id = row['district_id']
        try:
            params = {
                "lat" : lat,
                "lon" : lon,
                "appid": API_KEY,
                "units": "metric"
            }
            async with session.get(WEATHER_ENDPOINT, params=params) as response:
                # if response.status != 200:
                #     return {
                #         'province': province,
                #         'amphoe': amphoe,
                #         'lat': lat,
                #         'lon': lon,
                #         'error': f"HTTP {response.status}"
                #     }
                data = await response.json()

                
                weather_dict = {
                    'timestamp': dt,
                    'year': dt.year,
                    'month': dt.month,
                    'day': dt.day,
                    'hour': dt.hour,
                    'minute': dt.minute,
                    'localtime': localtime,
                    'province' : province,
                    'district' : district,
                    'district_id' : district_id,
                    'lat' : data['coord']['lat'],
                    'lon' : data['coord']['lon'], 
                    # "location": weather_data.get("name", district),
                    'weather_main' : data["weather"][0]["main"],
                    'weather_description' : data["weather"][0]["description"],
                    'main.temp' : data["main"]["temp"],
                    'main.temp_min' : data["main"]["temp_min"],
                    'main.temp_max' : data["main"]["temp_max"],
                    'main.feels_like' : data["main"]["feels_like"],
                    'main.pressure' : data["main"]["pressure"],
                    'main.humidity' : data["main"]["humidity"],
                    'visibility' : data.get("visibility"),
                    'wind.speed' : data["wind"]["speed"],
                    'wind.deg' : data["wind"]["deg"]
                    }
                return weather_dict
       
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None
        except KeyError as e:
            print(f"Error processing data: Missing key {e}")
            return None
        except Exception as e:
            print(f"Error: {e} at {province} - {district}")
            return None



    weather_results = []
    total_batches = ceil(len(df_sample) / batch_size)  #sample
    async with aiohttp.ClientSession() as session:
        for i in range(total_batches):
            batch = df_sample.iloc[i*batch_size:(i+1)*batch_size]
            tasks = [fetch_row(session, row) for _, row in batch.iterrows()]
            batch_results = await asyncio.gather(*tasks)
            weather_results.extend(batch_results)

            print(f"✅ เสร็จ batch {i+1}/{total_batches}")
            if i < total_batches - 1:
                await asyncio.sleep(65)  # ✅ รอให้ผ่าน rate limit

    return weather_results



def clean_data(df):
    df = pd.DataFrame(df)
    # if 'error' in df.columns:
    #     # df = df[df['error'].isna()]  # กรอง row ที่มี error ออก
    #     df = df.drop(columns=['error'], errors='ignore')  # ลบคอลัมน์ error
    df['province'] = df['province'].astype("string")
    df['district'] = df['district'].astype("string")
    return df


def save_to_lakefs_pollution(df):
    # ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
    # SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
    # lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")
    # ACCESS_KEY = "access_key"
    # SECRET_KEY = "secret_key"
    # lakefs_endpoint = "http://lakefs-dev:8000/"

    repo = "pollution-data"
    branch = "main"
    path = "pollution.parquet"
    
    lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

    storage_options = {
        "key": ACCESS_KEY,
        "secret": SECRET_KEY,
        "client_kwargs": {"endpoint_url": lakefs_endpoint}
    }

    df.to_parquet(
        lakefs_s3_path,
        storage_options=storage_options,
        partition_cols=["year", "month", "day", "hour"],
    )


def save_to_lakefs_weather(df):
    # ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
    # SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
    # lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")
    # ACCESS_KEY = "access_key"
    # SECRET_KEY = "secret_key"
    # lakefs_endpoint = "http://lakefs-dev:8000/"

    repo = "weather-data"
    branch = "main"
    path = "weather.parquet"
    
    lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

    storage_options = {
        "key": ACCESS_KEY,
        "secret": SECRET_KEY,
        "client_kwargs": {"endpoint_url": lakefs_endpoint}
    }

    df.to_parquet(
        lakefs_s3_path,
        storage_options=storage_options,
        partition_cols=["year", "month", "day", "hour"],
    )



### FLOW

In [6]:
import time
import os
import pandas as pd

@flow(name="main-flow", log_prints=True)
async def main_flow():
    start_time = time.perf_counter()  # ⏱ เริ่มจับเวลา

    dt = datetime.utcnow()
    thai_tz = pytz.timezone('Asia/Bangkok')
    localtime = dt.astimezone(thai_tz)
    
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work
    coord_path = os.path.join(BASE_DIR, "save", "district_coord.csv")
    coord_df = pd.read_csv(coord_path)
    df_sample = coord_df.sample(10)
    
    # pollution_results = await fetch_pollution_data(coord_df)
    pollution_results = await fetch_pollution_data(df_sample, dt, localtime)
    weather_results = await fetch_weather_data(df_sample, dt, localtime)
    # weather_data = pd.DataFrame(weather_results)
    # print(weather_data)
    # #Check Error
    # df_raw = pd.DataFrame(pollution_results)
    # if 'error' in df_raw.columns:
    #     error_df = df_raw[df_raw['error'].notna()]
    #     print("\n❗ Error Rows:")
    #     print(error_df[['province', 'amphoe', 'error']].head())
    
    pollution_data = clean_data(pollution_results)
    weather_data = clean_data(weather_results)
    
    end_time = time.perf_counter()  # ⏱ จับเวลาอีกครั้ง
    print(f"\n✅ ดึงข้อมูลเสร็จทั้งหมด ใช้เวลา {end_time - start_time:.2f} วินาที")
    save_to_lakefs_pollution(pollution_data)
    save_to_lakefs_weather(weather_data)
    # print("save to Lakefs Success")
    # print(pollution_data.head())
    # print(weather_data.head())
    # return pollution_data


In [8]:
#Check path + Sample
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work
coord_path = os.path.join(BASE_DIR, "save", "district_coord.csv")
print(coord_path)
coord_df = pd.read_csv(coord_path)
df_sample = coord_df.sample(10)
df_sample

/home/jovyan/save/district_coord.csv


,province_id,province_th,province_en,district_id,district_th,district_en,lat,lon
834,15,อ่างทอง,Ang Thong,1504,โพธิ์ทอง,Pho Thong,14.667000,100.417000
313,12,นนทบุรี,Nonthaburi,1206,ปากเกร็ด,Pak Kret,13.907700,100.495900
441,93,พัทลุง,Phatthalung,9306,ปากพะยูน,Pak Phayun,7.296220,100.295420
289,80,นครศรีธรรมราช,Nakhon Si Thammarat,8015,ขนอม,Khanom,9.204722,99.861111
284,80,นครศรีธรรมราช,Nakhon Si Thammarat,8020,พระพรหม,Phra Phrom,8.338333,99.901667
639,90,สงขลา,Songkhla,9003,จะนะ,Chana,6.895840,100.701260
127,22,จันทบุรี,Chanthaburi,2209,นายายอาม,Na Yai Am,12.738330,101.875020
426,56,พะเยา,Phayao,5603,เชียงคำ,Chiang Muan,19.506400,100.394700
541,45,ร้อยเอ็ด,Roi Et,4509,หนองพอก,Nong Phok,16.303570,104.221440
291,80,นครศรีธรรมราช,Nakhon Si Thammarat,8018,ถ้ำพรรณรา,Tham Phannara,8.420000,99.395000


### TEST

In [7]:
# test-run main flow
import nest_asyncio
nest_asyncio.apply()

await main_flow()


09:36:57.919 | INFO    | prefect.engine - Created flow run 'steady-beluga' for flow 'main-flow'

09:36:57.923 | INFO    | prefect.engine - View at http://prefect-server:4200/runs/flow-run/0f896dcd-576e-450d-bd4a-588fdcc06e6e

09:36:58.171 | INFO    | Task run 'fetch_pollution_data-0' - Created task run 'fetch_pollution_data-0' for task 'fetch_pollution_data'

09:36:58.602 | INFO    | Task run 'fetch_pollution_data-0' - ✅ เสร็จ batch 1/1

09:36:58.644 | INFO    | Task run 'fetch_pollution_data-0' - Finished in state Completed()

09:36:58.766 | INFO    | Task run 'fetch_weather_data-0' - Created task run 'fetch_weather_data-0' for task 'fetch_weather_data'

09:36:59.016 | INFO    | Task run 'fetch_weather_data-0' - ✅ เสร็จ batch 1/1

09:36:59.039 | INFO    | Task run 'fetch_weather_data-0' - Finished in state Completed()

09:36:59.048 | INFO    | Flow run 'steady-beluga' - 
✅ ดึงข้อมูลเสร็จทั้งหมด ใช้เวลา 1.06 วินาที

09:37:00.524 | INFO    | Flow run 'steady-beluga' - Finished in state Completed()

In [ ]:
def clean_data(results):
    # pollution_result = pd.DataFrame(results)
    pollution_result = pollution_result[pollution_result['error'].isna()]  # กรอง error ออก
    return pollution_result.drop(columns=['error'], errors='ignore')

# แยก clean กับ error ออก
clean_results = [r for r in results if 'error' not in r]
error_results = [r for r in results if 'error' in r]

# สร้าง DataFrame
clean_df = pd.DataFrame(clean_results)
error_df = pd.DataFrame(error_results)

# Save ไฟล์ clean ขึ้น LakeFS (หรือ local เตรียมอัป)
clean_df.to_parquet("pollution_data.parquet", index=False)

# เก็บ error ไว้ดูภายหลัง (optional)
if not error_df.empty:
    error_df.to_csv("pollution_errors.csv", index=False)



In [ ]:
@task
async def fetch_weather_data(lat, lon, timestamp):
    # ใช้ timestamp เดียวกันที่ส่งเข้ามา
    ...
    return {
        "timestamp": timestamp,
        "temperature": ...,
        "humidity": ...,
        ...
    }

@task
async def fetch_pollution_data(lat, lon, timestamp):
    ...
    return {
        "timestamp": timestamp,
        "pm25": ...,
        "pm10": ...,
        ...
    }


In [ ]:
@flow
async def combined_flow():
    dt = datetime.utcnow()
    # หรือ localtime ถ้าจะทำให้ consistent ตามประเทศไทย
    timestamp = dt.replace(second=0, microsecond=0)  

    weather_data = await fetch_weather_data(lat, lon, timestamp)
    pollution_data = await fetch_pollution_data(lat, lon, timestamp)

    # รวมข้อมูลจากทั้ง 2 API
    combined = {**weather_data, **pollution_data}
    df = pd.DataFrame([combined])
    return df
